Extend Jooha's code
1. Select X
2. Select Y
3. Select according to which X and Y you chose
4. Results

In [1]:
# Importing necessary Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
# Packages for reinforcement learning
import keras
from keras import layers, models, optimizers
from keras import backend as K
from collections import namedtuple, deque
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

# Packages/modules for data processing and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
import datetime
import math
from numpy.random import choice
import random
from collections import deque

Using TensorFlow backend.


Please download the original file converted to csv.

In [2]:
# Loading Dataset
data = pd.read_csv("MBAselection.csv")

In [3]:
# shape
data.shape

(391, 26)

In [4]:
# peek at data
set_option('max_columns', None)
data.head()

,SlNo,Gender,Gender-B,Percent_SSC,Board_SSC,Board_CBSE,Board_ICSE,Percent_HSC,Board_HSC,Stream_HSC,Percent_Degree,Course_Degree,Degree_Engg,Experience_Yrs,Entrance_Test,S-TEST,Percentile_ET,Percent_MBA,S-TEST*SCORE,Specialization_MBA,Marks_Communication,Marks_Projectwork,Marks_BOCA,Placement,Placement_B,Salary
0,1,M,0,62.00,Others,0,0,88.00,Others,Commerce,52.00,Science,0,0,MAT,1,55.0,58.80,55.0,Marketing & HR,50,65,74,Placed,1,270000
1,2,M,0,76.33,ICSE,0,1,75.33,Others,Science,75.48,Computer Applications,0,1,MAT,1,86.5,66.28,86.5,Marketing & Finance,69,70,75,Placed,1,200000
2,3,M,0,72.00,Others,0,0,78.00,Others,Commerce,66.63,Engineering,1,0,NaN,0,0.0,52.91,0.0,Marketing & Finance,50,61,59,Placed,1,240000
3,4,M,0,60.00,CBSE,1,0,63.00,CBSE,Arts,58.00,Management,0,0,MAT,1,75.0,57.80,75.0,Marketing & Finance,54,66,62,Placed,1,250000
4,5,M,0,61.00,CBSE,1,0,55.00,ISC,Science,54.00,Engineering,1,1,MAT,1,66.0,59.43,66.0,Marketing & HR,52,65,67,Placed,1,180000


In [5]:
print(data.Gender.unique())
print(data.Board_SSC.unique())
print(data.Board_HSC.unique())
print(data.Stream_HSC.unique())
print(data.Course_Degree.unique())
print(data.Entrance_Test.unique())
print(data.Specialization_MBA.unique())
print(data.Placement.unique())

#print(data['Gender'].value_counts())
#print(data['Board_SSC'].value_counts())
#print(data['Board_HSC'].value_counts())
#print(data['Stream_HSC'].value_counts())
#print(data['Course_Degree'].value_counts())
#print(data['Entrance_Test'].value_counts())
#print(data['Specialization_MBA'].value_counts())
#print(data['Placement'].value_counts())

['M' 'F']
['Others' 'ICSE' 'CBSE']
['Others' 'CBSE' 'ISC']
['Commerce' 'Science' 'Arts']
['Science' 'Computer Applications' 'Engineering' 'Management' 'Commerce'
 'Others' 'Arts']
['MAT' nan 'K-MAT' 'CAT' 'PGCET' 'GCET' 'G-MAT' 'XAT' 'G-SAT']
['Marketing & HR' 'Marketing & Finance' 'Marketing & IB']
['Placed' 'Not Placed']


In [6]:
data["Board_SSC"] = data["Board_SSC"].replace({'CBSE':1, 'ICSE':2, 'Others':3})
data["Board_HSC"] = data["Board_HSC"].replace({'CBSE':1, 'ISC':2, 'Others':3})
data["Stream_HSC"] = data["Stream_HSC"].replace({'Commerce':1, 'Science':2, 'Arts':3})
data["Course_Degree"] = data["Course_Degree"].replace({'Science':1, 'Computer Applications':2, 'Engineering':3,
                                                       'Management':4, 'Commerce':5, 'Others':6, 'Arts':7})
data["Entrance_Test"] = data["Entrance_Test"].replace({'MAT':1, 'K-MAT':2, 'CAT': 3, 'PGCET':4,
                                                       'GCET':5, 'G-MAT':6, 'XAT':7, 'G-SAT':8})
data["Specialization_MBA"] = data["Specialization_MBA"].replace({'Marketing & HR':1,
                                                                 'Marketing & Finance':2,
                                                                 'Marketing & IB':3})

In [7]:
data = data.drop("Gender", axis=1)
data = data.drop("Placement", axis=1)

data = data.drop("Board_CBSE", axis=1)
data = data.drop("Board_ICSE", axis=1)
data = data.drop("Degree_Engg", axis=1)

In [8]:
# shape
data.shape

(391, 21)

In [9]:
# peek at data
set_option('max_columns', None)
data.head(10)

,SlNo,Gender-B,Percent_SSC,Board_SSC,Percent_HSC,Board_HSC,Stream_HSC,Percent_Degree,Course_Degree,Experience_Yrs,Entrance_Test,S-TEST,Percentile_ET,Percent_MBA,S-TEST*SCORE,Specialization_MBA,Marks_Communication,Marks_Projectwork,Marks_BOCA,Placement_B,Salary
0,1,0,62.00,3,88.00,3,1,52.00,1,0,1.0,1,55.00,58.80,55.00,1,50,65,74,1,270000
1,2,0,76.33,2,75.33,3,2,75.48,2,1,1.0,1,86.50,66.28,86.50,2,69,70,75,1,200000
2,3,0,72.00,3,78.00,3,1,66.63,3,0,NaN,0,0.00,52.91,0.00,2,50,61,59,1,240000
3,4,0,60.00,1,63.00,1,3,58.00,4,0,1.0,1,75.00,57.80,75.00,2,54,66,62,1,250000
4,5,0,61.00,1,55.00,2,2,54.00,3,1,1.0,1,66.00,59.43,66.00,1,52,65,67,1,180000
5,6,0,55.00,2,64.00,1,1,50.00,5,0,NaN,0,0.00,56.81,0.00,2,53,70,53,1,300000
6,7,1,70.00,3,54.00,3,2,65.00,6,2,NaN,0,0.00,59.80,0.00,1,63,56,50,1,260000
7,8,0,68.00,2,77.00,2,1,72.50,4,0,1.0,1,43.12,57.23,43.12,2,74,72,50,1,235000
8,9,0,82.80,1,70.60,1,1,69.30,5,0,1.0,1,96.80,55.50,96.80,2,65,76,70,1,425000
9,10,1,59.00,1,74.00,1,3,59.00,4,1,NaN,0,0.00,63.83,0.00,1,50,59,77,1,240000


# 1. Select X
If you want to **include** either of Course_Degree, Entrance_Test, S-Test, or Specialization_MBA, please **add #** before the code enclosed in ### and ###.

In [10]:
#Generating Matrix of Features
data_X = data
data_X = data_X.drop("SlNo", axis=1)
data_X = data_X.drop("Placement_B", axis=1)
data_X = data_X.drop("Salary", axis=1)

###
# Note that this code does not add columns to the data, but drops columns
data_X = data_X.drop("Course_Degree", axis=1)
data_X = data_X.drop("Entrance_Test", axis=1)
data_X = data_X.drop("S-TEST", axis=1)
data_X = data_X.drop("Specialization_MBA", axis=1)
###

X = data_X.iloc[:].values
X[0:5]

array([[ 0.  , 62.  ,  3.  , 88.  ,  3.  ,  1.  , 52.  ,  0.  , 55.  ,
        58.8 , 55.  , 50.  , 65.  , 74.  ],
       [ 0.  , 76.33,  2.  , 75.33,  3.  ,  2.  , 75.48,  1.  , 86.5 ,
        66.28, 86.5 , 69.  , 70.  , 75.  ],
       [ 0.  , 72.  ,  3.  , 78.  ,  3.  ,  1.  , 66.63,  0.  ,  0.  ,
        52.91,  0.  , 50.  , 61.  , 59.  ],
       [ 0.  , 60.  ,  1.  , 63.  ,  1.  ,  3.  , 58.  ,  0.  , 75.  ,
        57.8 , 75.  , 54.  , 66.  , 62.  ],
       [ 0.  , 61.  ,  1.  , 55.  ,  2.  ,  2.  , 54.  ,  1.  , 66.  ,
        59.43, 66.  , 52.  , 65.  , 67.  ]])

# 2. Select Y

In [11]:
#Generating Dependent Variable Vectors
# If you want Y to be Placement_B
Y = data.iloc[:,-2].values
# If you want Y to be Salary
#Y = data.iloc[:,-1].values
Y[0:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [12]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [13]:
#Performing Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
#Initialising ANN
ann = tf.keras.models.Sequential()

In [15]:
 #Adding First Hidden Layer
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

In [16]:
 #Adding Second Hidden Layer
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

In [17]:
 #Adding Output Layer
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [18]:
#Compiling ANN
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
#X[0:5]

In [19]:
#Fitting ANN
ann.fit(X_train,Y_train,batch_size=1024,epochs = 100)

Train on 312 samples
Epoch 1/100
312/312 [==============================] - 1s 2ms/sample - loss: 0.6010 - accuracy: 0.7885
Epoch 2/100
312/312 [==============================] - 0s 10us/sample - loss: 0.5991 - accuracy: 0.8045
Epoch 3/100
312/312 [==============================] - 0s 10us/sample - loss: 0.5972 - accuracy: 0.8013
Epoch 4/100
312/312 [==============================] - 0s 6us/sample - loss: 0.5953 - accuracy: 0.8013
Epoch 5/100
312/312 [==============================] - 0s 10us/sample - loss: 0.5934 - accuracy: 0.8013
Epoch 6/100
312/312 [==============================] - 0s 6us/sample - loss: 0.5916 - accuracy: 0.8013
Epoch 7/100
312/312 [==============================] - 0s 10us/sample - loss: 0.5897 - accuracy: 0.7981
Epoch 8/100
312/312 [==============================] - 0s 10us/sample - loss: 0.5879 - accuracy: 0.7917
Epoch 9/100
312/312 [==============================] - 0s 16us/sample - loss: 0.5861 - accuracy: 0.7917
Epoch 10/100
312/312 [========================

Epoch 80/100
312/312 [==============================] - 0s 16us/sample - loss: 0.4902 - accuracy: 0.8141
Epoch 81/100
312/312 [==============================] - 0s 6us/sample - loss: 0.4893 - accuracy: 0.8141
Epoch 82/100
312/312 [==============================] - 0s 10us/sample - loss: 0.4885 - accuracy: 0.8141
Epoch 83/100
312/312 [==============================] - 0s 10us/sample - loss: 0.4876 - accuracy: 0.8141
Epoch 84/100
312/312 [==============================] - 0s 13us/sample - loss: 0.4867 - accuracy: 0.8141
Epoch 85/100
312/312 [==============================] - 0s 11us/sample - loss: 0.4858 - accuracy: 0.8141
Epoch 86/100
312/312 [==============================] - 0s 10us/sample - loss: 0.4849 - accuracy: 0.8141
Epoch 87/100
312/312 [==============================] - 0s 10us/sample - loss: 0.4840 - accuracy: 0.8141
Epoch 88/100
312/312 [==============================] - 0s 10us/sample - loss: 0.4832 - accuracy: 0.8141
Epoch 89/100
312/312 [==============================] - 

In [20]:
#print(data['Percent_SSC'].describe())
#print(data['Percent_HSC'].describe())
#print(data['Percent_Degree'].describe())
#print(data['Experience_Yrs'].describe())
#print(data['S-TEST'].describe())
#print(data['Percentile_ET'].describe())
#print(data['Percent_MBA'].describe())
#print(data['S-TEST*SCORE'].describe())
#print(data['Marks_Communication'].describe())
#print(data['Marks_Projectwork'].describe())
#print(data['Marks_BOCA'].describe())
#print(data['Salary'].describe())

### X
- **Gender_B:** {'M':0, 'F':1}
- **Percent_SSC:**
 <br>mean: 64.652353, min: 37.000000, 50%: 64.500000, max: 87.200000
- **Board_SSC:** {'CBSE':1, 'ICSE':2, 'Others':3}
- **Percent_HSC:**
 <br>mean  63.795294, min: 40.000000, 50%: 63.000000, max: 94.700000
- **Board_HSC:** {'CBSE':1, 'ISC':2, 'Other':3}
- **Stream_HSC:** {'Commerce':1, 'Science':2, 'Arts':3}
- **Percent_Degree:**
 <br>mean: 62.983606, min  35.000000, 50%: 63.000000, max: 89.000000
- **Course_Degree:** {'Science':1, 'Computer Applications':2, 'Engineering':3, 'Management':4, 'Commerce':5, 'Others':6, 'Arts':7}
- **Experience_Yrs:**
 <br>mean:  0.478261, min:  0.000000, 50%:  0.000000, max:  3.000000
- **Entrance_Test:** {'MAT':1, 'K-MAT':2, 'CAT': 3, 'PGCET':4, 'GCET':5, 'G-MAT':6, 'XAT':7, 'G-SAT':8}
- **S-TEST:**
 <br>mean:  0.828645, min:  0.000000, 50%:  1.000000, max:  1.000000
- **Percentile_ET:**
 <br>mean: 54.934962, min:  0.000000, 50%: 62.000000, max: 98.690000
- **Percent_MBA:**
 <br>mean: 61.667698, min: 50.830000, 50%: 61.010000, max: 77.890000
- **S-TEST SCORE:**
 <br>mean: 54.934962, min:  0.000000, 50%: 62.000000, max: 98.690000
- **Specialization_MBA:** {'Marketing & HR':1, 'Marketing & Finance':2, 'Marketing & IB':3}
- **Marks_Communication:**
 <br>mean: 60.539642, min: 50.000000, 50%: 58.000000, max: 88.000000
- **Marks_Projectwork:**
 <br>mean: 68.363171, min: 50.000000, 50%: 69.000000, max: 87.000000
- **Marks_BOCA:**
 <br>mean: 64.378517, min: 50.000000, 50%: 63.000000, max: 96.000000

### Y
- **Placement_B:** {'Not Placed':0, 'Placed':1}<br>
or
- **Salary:**
 <br>mean: 219078.260870, min: 0.000000, 50%: 240000.000000, max: 940000.000000


# 3. Select according to which X and Y you chose

In [21]:
#Predicting result for Single Observation
#print()
print(ann.predict(sc.transform([[0, 62, 3, 88, 3, 1, 52, 0, 55, 58.8, 55, 50, 65, 74]])) > 0.5)
#print(ann.predict(sc.transform([[0, 62, 3, 88, 3, 1, 52, 0, 55, 58.8, 55, 50, 65, 74]])) > 200000)
#print(ann.predict(sc.transform([[0, 62, 3, 88, 3, 1, 52, 1, 0, 1, 1, 55, 58.8, 55, 1, 50, 65, 74]])) > 0.5)
#print(ann.predict(sc.transform([[0, 62, 3, 88, 3, 1, 52, 1, 0, 1, 1, 55, 58.8, 55, 1, 50, 65, 74]])) > 200000)

[[ True]]


# 4. Results
- **Default**
 - X: Gender-B, Percent_SSC, Board_SSC, Percent_HSC, Board_HSC, Stream_HSC, Percent_Degree, Course_Degree, Experience_Yrs, Entrance_Test, S-TEST, Percentile_ET, Percent_MBA, S-TEST SCORE, Specialization_MBA, Marks_Communication, Marks_Projectwork, Marks_BOCA
 - Y: Placement_B


 - **Case 1: Default**<br>
Epoch 1/100
312/312 [==============================] - 0s 2ms/sample - loss: 0.6817 - accuracy: 0.4615<br>
Epoch 50/100
312/312 [==============================] - 0s 19us/sample - loss: 0.5752 - accuracy: 0.7628<br>
Epoch 100/100
312/312 [==============================] - 0s 6us/sample - loss: 0.5203 - accuracy: 0.8141<br>
**Good result.**


 - **Case 2: Change Y to "Salary"**<br>
Epoch 1/100
312/312 [==============================] - 0s 1ms/sample - loss: 126637.0234 - accuracy: 0.1795<br>
Epoch 50/100
312/312 [==============================] - 0s 10us/sample - loss: 30238.7363 - accuracy: 0.0769<br>
Epoch 100/100
312/312 [==============================] - 0s 10us/sample - loss: -56825.7812 - accuracy: 0.0096<br>
**Accuracy has dropped significantly.**

 
 - **Case 3: Add all "Course_Degree", "Entrance_Test", "S-TEST", "Specialization_MBA" to default X (Y is Placement_B)**<br>
Epoch 1/100
312/312 [==============================] - 0s 2ms/sample - loss: nan - accuracy: 0.6667<br>
Epoch 50/100
312/312 [==============================] - 0s 13us/sample - loss: nan - accuracy: 0.1859<br>
Epoch 100/100
312/312 [==============================] - 0s 6us/sample - loss: nan - accuracy: 0.1859<br>
**Accuracy has dropped significantly.**


 - **Case 4: Add "Course_Degree" to default X (Y is Placement_B)**<br>
Epoch 1/100
312/312 [==============================] - 1s 2ms/sample - loss: 0.6066 - accuracy: 0.7147<br>
Epoch 50/100
312/312 [==============================] - 0s 13us/sample - loss: 0.5176 - accuracy: 0.8141<br>
Epoch 100/100
312/312 [==============================] - 0s 10us/sample - loss: 0.4628 - accuracy: 0.8141<br>
**Good result.**


 - **Case 5: Add "Entrance_Test" to default X (Y is Placement_B)**<br>
Epoch 1/100
312/312 [==============================] - 1s 2ms/sample - loss: nan - accuracy: 0.7179<br>
Epoch 50/100
312/312 [==============================] - 0s 10us/sample - loss: nan - accuracy: 0.1859<br>
Epoch 100/100
312/312 [==============================] - 0s 13us/sample - loss: nan - accuracy: 0.1859<br>
**Accuracy has dropped significantly.**


 - **Case 6: Add "S-TEST" to default X (Y is Placement_B)**<br>
Epoch 1/100
312/312 [==============================] - 1s 2ms/sample - loss: 0.7427 - accuracy: 0.6058<br>
Epoch 50/100
312/312 [==============================] - 0s 13us/sample - loss: 0.5891 - accuracy: 0.7404<br>
Epoch 100/100
312/312 [==============================] - 0s 13us/sample - loss: 0.4994 - accuracy: 0.8141<br>
**Good result.**


 - **Case 7: Add "Specialization_MBA" to default X (Y is Placement_B)**<br>
Epoch 1/100
312/312 [==============================] - 1s 2ms/sample - loss: 0.6284 - accuracy: 0.6186<br>
Epoch 50/100
312/312 [==============================] - 0s 10us/sample - loss: 0.5322 - accuracy: 0.7981<br>
Epoch 100/100
312/312 [==============================] - 0s 10us/sample - loss: 0.4749 - accuracy: 0.8141<br>
**Good result.**

 - **Case 8: Add "Course_Degree", "S-TEST", "Specialization_MBA" to default X, i.e. exclude "Entrance_Test" (Y is Placement_B)**<br>
Epoch 1/100
312/312 [==============================] - 1s 2ms/sample - loss: 0.7752 - accuracy: 0.2244<br>
Epoch 50/100
312/312 [==============================] - 0s 10us/sample - loss: 0.6684 - accuracy: 0.6506<br>
Epoch 100/100
312/312 [==============================] - 0s 13us/sample - loss: 0.5953 - accuracy: 0.8141<br>
**Good result.**

- **Conclusion**<br>
In this extension, we tried the above eight cases. It is considered better to analyze with the following variables:
 - X: Gender-B, Percent_SSC, Board_SSC, Percent_HSC, Board_HSC, Stream_HSC, Percent_Degree, Course_Degree, Experience_Yrs, S-TEST, Percentile_ET, Percent_MBA, S-TEST SCORE, Specialization_MBA, Marks_Communication, Marks_Projectwork, Marks_BOCA<br>
 - Y: Placement_B

 When "Salary" was set to Y, the accuracy dropped. This may be due to the fact that "Salary" is dependent on the factors in X.<br>
 When "Entrance_Test" was added to X, the accuracy dropped. This may be due to the fact that "Entrance_Test" consists of many dummy variables.